In [0]:
# -*- coding: utf-8 -*-
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd, numpy as np
import cvxpy

# Read recipe inputs
nba_knapsack_prep = dataiku.Dataset("nba_knapsack_prep")
knapsack_df = nba_knapsack_prep.get_dataframe()

### Define Parameters

In [0]:
agg_col = 'Date'
value_col = 'Points_Predict'
cost_col = 'Salary_DK'
label_col = 'Name_BBRef'
actual_col = 'Points_Actual'
cap = 50000
top_n = 3
selection_n = 8

In [0]:
agg_distinct = list(knapsack_df[agg_col].unique())
agg_distinct.sort()

In [0]:
# Inspiration from: https://towardsdatascience.com/integer-programming-in-python-1cbdfa240df2

def knapsack(df, label_col, value_col, cost_col, selection_n, max_val):

    # gather elements to select from
    costs = np.array(df[cost_col])
    values = np.array(df[value_col])
    selection = cvxpy.Variable(len(costs), boolean=True)

    # objective function (to maximize)
    total_value = values @ selection

    # universal constraints (weight constraints and constraint on number of selections)
    cost_constraint = costs @ selection <= cap
    roster_constraint = np.ones(len(costs)) @ selection == selection_n
    max_constraint = values @ selection <= max_val # max val allows us to get "n best" optimal values

    # specific constraints (custom constraints for situation)
    pg = np.array(df['PG'])
    sg = np.array(df['SG'])
    sf = np.array(df['SF'])
    pf = np.array(df['PF'])
    c = np.array(df['C'])
    pg_constraint = pg @ selection >= 1
    sg_constraint = sg @ selection >= 1
    sf_constraint = sf @ selection >= 1
    pf_constraint = pf @ selection >= 1
    c_constraint = c @ selection >= 1
    g_constraint = (pg + sg) @ selection >= 3
    f_constraint = (sf + pf) @ selection >= 3

    knapsack_problem = cvxpy.Problem(cvxpy.Maximize(total_value), [cost_constraint, roster_constraint, max_constraint,
                                                                   pg_constraint, sg_constraint, sf_constraint,
                                                                   pf_constraint, c_constraint, g_constraint, f_constraint])
    value_opt = knapsack_problem.solve(solver=cvxpy.GLPK_MI)
    selection_opt = np.array(selection.value).astype(int)
    total_cost = (costs @ selection).value

    return selection_opt, value_opt, total_cost

In [0]:
output_data = []
for agg_bin in agg_distinct:
    print(agg_bin)
    max_val = np.inf
    for iteration in range(top_n):
        knapsack_bin_df = knapsack_df.loc[knapsack_df[agg_col] == agg_bin]
        selection_opt, value_opt, total_cost = knapsack(knapsack_bin_df, label_col, value_col, cost_col, selection_n, max_val)

        labels, actuals = list(knapsack_bin_df[label_col]), list(knapsack_bin_df[actual_col])
        labels_opt, actuals_opt = [], []
        for i in range(len(selection_opt)):
            if selection_opt[i]:
                labels_opt.append(labels[i])
                actuals_opt.append(actuals[i])
        output_row = [agg_bin]
        output_row.extend(labels_opt)
        output_row.extend([int(total_cost), np.round(value_opt, 3), np.sum(actuals_opt)])
        output_data.append(output_row)
        max_val = value_opt - 10**-3

In [0]:
output_cols = [agg_col]
output_cols.extend(range(1, selection_n+1))
output_cols.extend(['Total_Cost', 'Value_Predict', 'Value_Actual'])

In [0]:
nba_knapsack_output_df = pd.DataFrame(output_data, columns=output_cols)
nba_knapsack_output_df

In [0]:
# Write recipe outputs
nba_knapsack_output = dataiku.Dataset("nba_knapsack_output")
nba_knapsack_output.write_with_schema(nba_knapsack_output_df)